In [ ]:
#載入所需函示庫
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
# 設定圖形大小; DPI越大圖越大
plt.rcParams["figure.dpi"] = 100
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as smm
import statsmodels.stats.outliers_influence as sso
from statsmodels.stats.stattools import durbin_watson
import statsmodels
import statistics as stat
import statistics
import math
from sklearn.utils import shuffle
# !pip install stepwise-regression
from stepwise_regression import step_reg
import time
import itertools
from statsmodels.tsa.api import Holt

In [ ]:
!pip install stepwise-regression

In [ ]:
def my_inference(p_value, alpha, claim):
    if p_value < alpha:
        print(f"Since the p_value = {p_value:.4f} < {alpha}, we reject the null hypothesis.\nThat is, we have sufficient evidence to claim that {claim}.")
    else:
        print(f"Since the p_value = {p_value:.4f} > {alpha}, we do not reject the null hypothesis.\nThat is, we do not have sufficient evidence to claim that {claim}.")

def my_normality_check(sample_data, data_name, data_unit, alpha):
    # histogram
    fig, ax = plt.subplots()
    counts, bins, patches = plt.hist((sample_data.dropna()).astype(float), 6, density=False, facecolor='g', alpha=0.75)
    plt.title(f'Histogram of {data_name}')
    plt.xlabel(data_unit)
    plt.ylabel('Frequency')
    plt.grid(True)

    # Shapiro-Wilk test
    print("Shapiro-Wilk test for normality:")
    print("H0: The distribution is normal.")
    print("H1: The distribution is not normal.")
    stat, p = stats.shapiro(sample_data.dropna())
    print(f'For population = {data_name}')
    print(f"Shapiro statistic = {stat:.6f} and p_value = {p:.6f}")
    p_value = p; claim = "the distribution is not normal"
    my_inference(p_value, alpha, claim)

    # QQ plot
    fig = sm.qqplot(sample_data.dropna(), stats.norm, fit=True, line='45')
    plt.title(f'Q-Q Plot for {data_name}')
    plt.show()

def my_scatter_plot(sample_data, X_name, Y_name, X_unit, Y_unit):
    _ = sns.regplot(x = sample_data[X_name], y = sample_data[Y_name], color = 'b', ci = None)
    plt.title(f'Scatter Plot for {X_name} and {Y_name}')
    plt.xlabel(f'{X_name} ({X_unit})')
    plt.ylabel(f'{Y_name} ({Y_unit})')
    plt.show()

def my_scatter_plot_2(x, y):
    _ = sns.regplot(x = x, y = y, color = 'b', ci = None)
    plt.title(f'Scatter Plot for {x.name} and {y.name}')
    plt.xlabel(f'{x.name}')
    plt.ylabel(f'{y.name}')
    plt.show()

def Sample_Mean_Hypothesis_Testing(x, H0_x_bar, a):
    x_bar = x.mean()
    x_std = stat.stdev(x)
    x_n = x.size
    print(f"mean = {x_bar:.4f}")
    print(f"std. dev. = {x_std:.4f}")
    print(f"Number of observation = {x_n}")
    print(f"Hypothesized mean = {H0_x_bar}")
    print(f"Significant level = {a}")
    tstat = (x_bar - H0_x_bar) / (x_std / (x_n ** 0.5))
    print(f"t-stat = {tstat:.4f}")
    if tstat > 0:
        tcv_onetail = stats.t.ppf(1 - a, df = x_n - 1)
    else:
        tcv_onetail = stats.t.ppf(a, df = x_n - 1)
    print(f"t critical value one tail = {tcv_onetail:.4f}")
    if tstat > 0:
        p_onetail = 1 - stats.t.cdf(tstat, df = x_n - 1)
    else:
        p_onetail = stats.t.cdf(tstat, df = x_n - 1)
    print(f"p-value (one-tail) = {p_onetail:.4f}")
    if tstat > 0:
        tcv_twotail = stats.t.ppf(1 - a/2, df = x_n - 1)
    else:
        tcv_twotail = stats.t.ppf(a/2, df = x_n - 1)
    print(f"t critical value two tail = {tcv_twotail:.4f}")
    p_twotail = p_onetail * 2
    print(f"p-value (two-tail) = {p_twotail:.4f}")
    return p_onetail, p_twotail

def runsTest(l, l_median):
    runs, n1, n2 = 1, 0, 0
    if(l[0]) >= l_median:
        n1 += 1
    else:
        n2 += 1
    for i in range(1, len(l)):
        if (l[i] >= l_median and l[i-1] < l_median) or (l[i] < l_median and l[i-1] >= l_median):
            runs += 1
        if(l[i]) >= l_median:
            n1 += 1
        else:
            n2 += 1
    runs_exp = ((2*n1*n2)/(n1+n2)) + 1
    stan_dev = math.sqrt((2*n1*n2*(2*n1*n2-n1-n2))/(((n1+n2)**2)*(n1+n2-1)))
    z = (runs-runs_exp)/stan_dev
    pval_z = stats.norm.sf(abs(z)) * 2
    print('runs = ', runs)
    print('n1 = ', n1)
    print('n2 = ', n2)
    print('runs_exp = ', runs_exp)
    print('stan_dev = ', stan_dev)
    print('z = ', z)
    print('pval_z = ', pval_z)
    return pval_z

def Durbin_Watson_test(x):
    x_square_sum = np.vdot(x, x)
    print("x_square_sum = ", x_square_sum)
    size = x.size
    print("size = ", size)
    x_d = np.zeros((size))
    print("x_d = ", x_d)
    l_size = size - 1
    for i in range(l_size):
        x_d[i + 1] = x[i + 1] - x[i]
    print("x_d = ", x_d)
    d = np.vdot(x_d, x_d) / x_square_sum
    print("d = ", d)
    return(d)

def required_conditions_for_error(SD, y_pre, Y_name, d):
    # check required conditions
    print('Checking for required conditions for error variable:\n')

    mu = np.mean(SD)
    sigma = np.std(SD)

    ## The error is a random variable with mean of zero.
    print('\n1. Zero mean')
    print(f'H0: Errors have zero mean.')
    print(f'H1: Errors do not have zero mean.')
    H0_x_bar = 0
    p_onetail, p_twotail = Sample_Mean_Hypothesis_Testing(SD, H0_x_bar, alpha)
    claim = "the errors do not have zero mean"
    my_inference(p_twotail, alpha, claim)

    print(f'\n2. Normality')
    print(f'H0: Errors are normally distributed.')
    print(f'H1: Errors are not normally distributed.')


    ## The error is a normally distributed random variable.
    my_normality_check(pd.Series(SD), "standarized residuals", "unit", alpha)

    ## The variance of the error term is the same for all values of the independent variable.
    print('\n3. Homoskedasticity')
    print('H0: Homoskedasticity')
    print('H1: Heteroskedasticity')

    plt.plot(y_pre, SD, 'o', color = 'gray')
    plt.axhline(y=0, color = 'blue')
    plt.axhline(y=2, color = 'red')
    plt.axhline(y=-2, color = 'red')
    plt.title('Standardized Residual Plot')
    plt.xlabel(Y_name)
    plt.ylabel('Standardized Residual')
    plt.show()
    print('Do not rejected H0. Heteroscedasticity does not appear to be a problem.')


    ## The values of the error are independent
    print('\n4. Independence')
    print('\n4-1. Randomness')
    print('H0 : Randomness exists.')
    print('H1 : Randomness does not exist.')
    SD_median = statistics.median(SD)
    Z_pval = runsTest(SD, SD_median)
    print('p_value for Z-statistic= ', Z_pval)
    claim = 'randomness does not exist'
    my_inference(Z_pval, alpha, claim)

    print('\n4-2. No Autocorrelation')
    print('H0 : There is no first-order correlation.')
    print('H1 : There is first-order correlation.')
#     float(durbin_watson(results.resid))
    print(f'd = {d:.4f}')

def my_corr(y, X):
    data = pd.concat([y, X], axis=1)
    corr = data.corr()
    _ = sns.heatmap(corr, annot=True)
    multicollinearity_pairs = []
    for i in range(1, corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if (corr.iloc[i, j] > 0.7):
                multicollinearity_pairs.append((corr.columns[i], corr.columns[j]))
    corr_greater_than_07 = True
    if len(multicollinearity_pairs) == 0:
        print("No apparent multicollinearity problem of coefficients of correlation greater than 0.7.")
        corr_greater_than_07 = False
    else:
        print(f"Multicollinearity may exist between {multicollinearity_pairs} since their correlation of coefficient are greater than 0.7.")
    return corr, corr_greater_than_07

def my_outlier(SD):
    df = pd.DataFrame(SD,columns = ['SD'])
    filter = (df['SD'] < -2) | (df['SD'] > 2)
    outliers = df['SD'].loc[filter]
    if len(outliers) == 0:
        print('There are no outliers.')
    else:
        print("Outliers by SD = \n")
        print(outliers)
        print('\nOutliers:')
        out = ''
        indices = outliers.index
        for i, idx in enumerate(indices):
            if i == len(indices) - 1:
                out += f'sample {idx + 1}.'
            else:
                out += f'sample {idx + 1}, '
        print(out)

def my_hii(X, ols_result):
    H = np.matmul(X, np.linalg.solve(np.matmul(X.T, X), X.T))
    df_h = pd.DataFrame({
        'hii': np.diagonal(H)
    })
    k = ols_result.df_model
    n = len(df_h['hii'])
    h_level = 3 * (k + 1) / n
    print("h_level = ", h_level)
    print(" \n")
    filter = (df_h['hii'] > h_level )
    hii = df_h['hii']
    inf_hii = hii.loc[filter]
    if len(inf_hii) == 0:
        print('There are no influential observations by hii.')
    else:
        print("Influential observations by hii = \n")
        print(inf_hii)
        print('\Influential observations:')
        out = ''
        indices = inf_hii.index
        for i, idx in enumerate(indices):
            if i == len(indices) - 1:
                out += f'sample {idx + 1}.'
            else:
                out += f'sample {idx + 1}, '
    return hii, inf_hii


def my_cooks_distance(ols_result, data, hii):
    s2_e = ols_result.mse_resid
    k = ols_result.df_model
    y_a = data[:, 1]
    y_f = data[:, 2]
    CD_arr = np.square(y_a - y_f) / s2_e / (k - 1) * hii / np.square(1 - hii)
    CD = np.array(CD_arr)
    df_cd = pd.DataFrame(CD,columns = ['CD'])
    filter = (df_cd['CD'] > 1 )
    inf_cd = df_cd['CD'].loc[filter]

    if len(inf_cd) == 0:
        print("There are no influential observations by Cook's Distances.")
    else:
        print("Influential observations by Cook's Distances = \n")
        print(inf_cd)
        print('\Influential observations:')
        out = ''
        indices = inf_cd.index
        for i, idx in enumerate(indices):
            if i == len(indices) - 1:
                out += f'sample {idx + 1}.'
            else:
                out += f'sample {idx + 1}, '
    return inf_cd

def my_simple_linear_regression(sample_data, X_name, Y_name, X_unit, Y_unit, alpha):
    # scatter plot
    my_scatter_plot(sample_data, X_name, Y_name, X_unit, Y_unit)

    # ols regression
    ols_result = smf.ols(f'{Y_name} ~ {X_name}', data = sample_data).fit()
    display(ols_result.summary())
    b1 = ols_result.params[1]
    b0 = ols_result.params[0]
    print(f"Estimated model: {Y_name} = {b0:0.4f} + {b1:0.4f} {X_name}")

    ## interpret the model
    print(f'The intercept is b0 = {b0:0.4f}.')
    print(f'The slope of the line is b1 = {b1:0.4f}. For each additional {X_unit} on {X_name}, \
          {Y_name} decreases by an average of {b1:0.4f} {Y_unit}.')



    # check required conditions

    ## required conditions for error term
    st, data, ss = sso.summary_table(ols_result, alpha = alpha)
    SD = data[:, 10]
    y_pre = data[:, 2]

    required_conditions_for_error(SD, y_pre, Y_name)

    # detecting outliers
    my_outlier(SD)

    # influential observations
    ## hii
    hii, inf_hii = my_hii(sample_data.drop(sample_data.columns[0], axis = 1) , ols_result)

    ## cook's distance
    my_cooks_distance(ols_result, data, hii)



def my_multiple_linear_regression(y, X, alpha):
    ## no muliticollinearity
    multicollinearity_pairs = my_multicollinearity_test(y, X)

    # ols regression
    X = sm.add_constant(X)

    ols_result = sm.OLS(y, X).fit()
    print(ols_result.summary())

    # check required conditions

    ## required conditions for error term
    st, data, ss = sso.summary_table(ols_result, alpha = alpha)
    SD = data[:, 10]
    y_pre = data[:, 2]
    required_conditions_for_error(SD, y_pre, y.name)

    # detecting outliers
    my_outlier(SD)

    # influential observations
    ## hii
    hii, inf_hii = my_hii(X, ols_result)

    ## cook's distance
    my_cooks_distance(ols_result, data, hii)


    print('\nAssessing the model:')

    ## The Standard Error of Estimate
    print('\nAssessment 1: standard error of estimate')
    s2_e = ols_result.mse_resid
    print(f'MSE: {s2_e:.4f}')
    s_e = ols_result.mse_resid ** 0.5
    print(f'Standard errors: {s_e:.4f}')
    print(f"mean of y = {y.mean():.4f}")
    print(f"variance of y = {y.var(ddof=1):.4f}")
    print(f"standard deviation of y = {y.std(ddof=1):.4f}")

    ## The Coefficient of Determination
    print('\nAssessment 2: Coefficient of Determination')
    print(f"R-squared: {ols_result.rsquared}")
    print(f"Adjusted R-squared: {ols_result.rsquared_adj}")
    if abs(ols_result.rsquared - ols_result.rsquared_adj) > 0.06:
        print(f'Since the difference between R-squared and the adjusted R-squared is {abs(ols_result.rsquared - ols_result.rsquared_adj):.4f} > 0.06, there might be a problem of over-fitting.')
    else:
        print(f'Since the difference between R-squared and the adjusted R-squared is {abs(ols_result.rsquared - ols_result.rsquared_adj):.4f} <= 0.06, there will not be a problem of over-fitting.')

    ## The F-test of ANOVA
    f_res = ols_result.fvalue
    print("F value = ", f_res)
    MSE = ols_result.mse_resid
    df_model = ols_result.df_model
    df_error = ols_result.df_resid
    MSR = f_res * MSE
    SSR = MSR * df_model
    print("SSR = ", SSR, "df = ", df_model, "MSR = ", MSR)
    print("SSE = ", MSE * df_error, "df = ", df_error, "MSE = ", MSE)
    print("F = ", MSR / MSE)
    A = np.identity(len(ols_result.params))
    A = A[1:,:]
    print("F test = ", ols_result.f_test(A))
    p_value = ols_result.f_pvalue
    claim = 'the model is valid'
    my_inference(p_value, alpha, claim)
    return ols_result


def processSubset(y_v, X_v, feature_set):
    X_v_a = sm.add_constant(X_v[list(feature_set)])
    model = sm.OLS(y_v,X_v_a)
    regr = model.fit()
    RSS = regr.rsquared_adj
    return { "model":regr.model.exog_names, "RSS":RSS }

def getBest(y_g, X_g, k):
    tic = time.time()
    results = []
    for combo in itertools.combinations(X_g.columns, k):
        results.append(processSubset(y_g, X_g, combo))
    models = pd.DataFrame(results)
    best_model = models.loc[models['RSS'].argmax()]
    toc = time.time()
    return best_model

In [ ]:
class LAZY_MLR:
    def __init__(self, y, X, alpha):
        self.y = y
        self.X = X
        self.alpha = 0.05
        X = sm.add_constant(X)
        self.ols_result = sm.OLS(y, X).fit()
#         print(self.ols_result.summary())
        st, data, ss = sso.summary_table(self.ols_result, alpha = alpha)
        self.SD = data[:, 10]
        self.y_pre = data[:, 2]
        self.durbin_watson_value = float(durbin_watson(self.ols_result.resid))
    def get_formula(self):
        s = f"{y.name} = {self.ols_result.params[0]:.4f}"
        for i in range(1, len(self.ols_result.params)):
            s += f" + {self.ols_result.params[i]:.4f} {self.ols_result.params.index[i]}"
        print(s)
        return s

    def get_ols_result(self):
        print(self.ols_result.summary())

    def get_scatter_plots(self):
        for col in self.X.columns:
            my_scatter_plot_2(self.X[col], self.y)

    def get_corr(self):
        self.corr, self.corr_greater_than_07 = my_corr(self.y, self.X)

    def check_multicollinearity(self):
        if not hasattr(self, "corr"):
            self.get_corr()
        opposite_signs = False
        for i in range(1, self.corr.shape[0]):
            if self.ols_result.params[i] * self.corr[y.name].iloc[i] < 0:
                opposite_signs = True
                print(f"The coefficient of correlation of ({self.y.name}, {self.ols_result.params.index[i]}) is {self.corr.iloc[0, i]:.4f} and the estimated coeffiecient is {self.ols_result.params[i]:.4f}. They have opposite signs, which is probably because of the multicollinearity problems.")
        if not opposite_signs:
            print('No opposite signs.')
        if not self.corr_greater_than_07 and not opposite_signs:
            print('Multicollinearity will not be a problem.')
        else:
            print('Multicollinearity may be a problem.')


    def forward_stepwise(self):
        self.forward_select = step_reg.forward_regression(self.X, self.y, self.alpha, verbose=False)
        print(self.forward_select)


    def backward_stepwise(self):
        self.backward_select = step_reg.backward_regression(self.X, self.y, self.alpha, verbose=False)
        print(self.backward_select)

    def best_subset(self):
        models_best = pd.DataFrame(columns=["RSS", "model"])
        tic = time.time()
        for i in range(1, int(self.ols_result.df_model) + 1):
            models_best.loc[i] = getBest(self.y, self.X, i)
        toc = time.time()
        print("Total elapsed time:", (toc-tic), "seconds.")
        display(models_best)
#         Fb = models_best[models_best['RSS']==models_best.RSS.max()].index.values
#         print(models_best.loc[Fb[0], "model"].summary())

    def check_required_conditions_for_error(self):
        required_conditions_for_error(self.SD, self.y_pre, self.y.name, self.durbin_watson_value)

    def get_model_assessment(self):
        # assessing the model
        print('Assessing the model:\n')

        ## standard error of estimate
        print('Assessment 1: Standard error of estimate')
        s2_e = self.ols_result.mse_resid
        print(f'MSE: {s2_e:.4f}')
        s_e = self.ols_result.mse_resid ** 0.5
        print(f'Standard errors: {s_e:.4f}')
        print(f"mean of y = {self.y.mean():.4f}")
        print(f"variance of y = {self.y.var(ddof=1):.4f}")
        print(f"standard deviation of y = {self.y.std(ddof=1):.4f}")
        if s_e < 0.3 * self.y.mean():
            print("The standard errors are relatively small, indicating that the model's variation is small and the model is a good fit.")
        else:
            print("The standard errors are relatively big, indicating that the model's variation is big and the model is not a good fit.")

        ## The Coefficient of Determination
        print('\nAssessment 2: Coefficient of Determination')
        print(f"R-squared: {self.ols_result.rsquared}")
        print(f"Adjusted R-squared: {self.ols_result.rsquared_adj}")
        if abs(self.ols_result.rsquared - self.ols_result.rsquared_adj) > 0.06:
            print(f'Since the difference between R-squared and the adjusted R-squared is {abs(self.ols_result.rsquared - self.ols_result.rsquared_adj):.4f} > 0.06, there might be a problem of over-fitting.')
        else:
            print(f'Since the difference between R-squared and the adjusted R-squared is {abs(self.ols_result.rsquared - self.ols_result.rsquared_adj):.4f} <= 0.06, there will not be a problem of over-fitting.')

        ## The F-test of ANOVA
        print('\nAssessment 3: F-test of ANOVA')
        print("H0: All of the estimated coefficients are zero.")
        print("H1: Not all of the estimated coefficients are zero.")
        f_res = self.ols_result.fvalue
        print(f"F value = {f_res:.4f}")
        MSE = self.ols_result.mse_resid
        df_model = self.ols_result.df_model
        df_error = self.ols_result.df_resid
        MSR = f_res * MSE
        SSR = MSR * df_model
        print("SSR = ", SSR, "df = ", df_model, "MSR = ", MSR)
        print("SSE = ", MSE * df_error, "df = ", df_error, "MSE = ", MSE)
        print("F = ", MSR / MSE)
        A = np.identity(len(self.ols_result.params))
        A = A[1:,:]
        print("F test = ", self.ols_result.f_test(A))
        p_value = self.ols_result.f_pvalue
        claim = 'the model is valid'
        my_inference(p_value, alpha, claim)

        # Testing of the Coefficients
        print('\nAssessment 4: t-test of the coefficients')
        for i in range(1, int(self.ols_result.df_model) + 1):
            print(f"H0: beta {i} = 0")
            print(f"H1: beta {i} ≠ 0")
            claim = f"the coefficient of {self.ols_result.params.index[i]} is not 0"
            my_inference(self.ols_result.pvalues[i], self.alpha, claim)

class LAZY_SLR:
    def __init__(self, y, x, alpha):
        self.y = y
        self.x = x
        self.alpha = 0.05
        x = sm.add_constant(x)
        self.ols_result = sm.OLS(y, x).fit()
#         print(self.ols_result.summary())
        st, data, ss = sso.summary_table(self.ols_result, alpha = alpha)
        self.SD = data[:, 10]
        self.y_pre = data[:, 2]
        self.durbin_watson_value = float(durbin_watson(self.ols_result.resid))
    def get_formula(self):
        s = f"{y.name} = {self.ols_result.params[0]:.4f}"
        for i in range(1, len(self.ols_result.params)):
            s += f" + {self.ols_result.params[i]:.4f} {self.ols_result.params.index[i]}"
        print(s)
        return s

    def get_ols_result(self):
        print(self.ols_result.summary())

    def get_scatter_plots(self):
        my_scatter_plot_2(self.x, self.y)

    def get_corr(self):
        self.corr, self.corr_greater_than_07 = my_corr(self.y, self.x)

#     def check_multicollinearity(self):
#         if not hasattr(self, "corr"):
#             self.get_corr()
#         opposite_signs = False
#         for i in range(1, self.corr.shape[0]):
#             if self.ols_result.params[i] * self.corr[y.name].iloc[i] < 0:
#                 opposite_signs = True
#                 print(f"The coefficient of correlation of ({self.y.name}, {self.ols_result.params.index[i]}) is {self.corr.iloc[0, i]:.4f} and the estimated coeffiecient is {self.ols_result.params[i]:.4f}. They have opposite signs, which is probably because of the multicollinearity problems.")
#         if not opposite_signs:
#             print('No opposite signs.')
#         if not self.corr_greater_than_07 and not opposite_signs:
#             print('Multicollinearity will not be a problem.')
#         else:
#             print('Multicollinearity may be a problem.')


#     def forward_stepwise(self):
#         self.forward_select = step_reg.forward_regression(self.x, self.y, self.alpha, verbose=False)
#         print(self.forward_select)


#     def backward_stepwise(self):
#         self.backward_select = step_reg.backward_regression(self.x, self.y, self.alpha, verbose=False)
#         print(self.backward_select)

#     def best_subset(self):
#         models_best = pd.DataFrame(columns=["RSS", "model"])
#         tic = time.time()
#         for i in range(1, int(self.ols_result.df_model) + 1):
#             models_best.loc[i] = getBest(self.y, self.X, i)
#         toc = time.time()
#         print("Total elapsed time:", (toc-tic), "seconds.")
#         display(models_best)
# #         Fb = models_best[models_best['RSS']==models_best.RSS.max()].index.values
# #         print(models_best.loc[Fb[0], "model"].summary())

    def check_required_conditions_for_error(self):
        required_conditions_for_error(self.SD, self.y_pre, self.y.name, self.durbin_watson_value)

    def get_model_assessment(self):
        # assessing the model
        print('Assessing the model:\n')

        ## standard error of estimate
        print('Assessment 1: Standard error of estimate')
        s2_e = self.ols_result.mse_resid
        print(f'MSE: {s2_e:.4f}')
        s_e = self.ols_result.mse_resid ** 0.5
        print(f'Standard errors: {s_e:.4f}')
        print(f"mean of y = {self.y.mean():.4f}")
        print(f"variance of y = {self.y.var(ddof=1):.4f}")
        print(f"standard deviation of y = {self.y.std(ddof=1):.4f}")
        if s_e < 0.3 * self.y.mean():
            print("The standard errors are relatively small, indicating that the model's variation is small and the model is a good fit.")
        else:
            print("The standard errors are relatively big, indicating that the model's variation is big and the model is not a good fit.")

        ## The Coefficient of Determination
        print('\nAssessment 2: Coefficient of Determination')
        print(f"R-squared: {self.ols_result.rsquared}")
        print(f"Adjusted R-squared: {self.ols_result.rsquared_adj}")
        if abs(self.ols_result.rsquared - self.ols_result.rsquared_adj) > 0.06:
            print(f'Since the difference between R-squared and the adjusted R-squared is {abs(self.ols_result.rsquared - self.ols_result.rsquared_adj):.4f} > 0.06, there might be a problem of over-fitting.')
        else:
            print(f'Since the difference between R-squared and the adjusted R-squared is {abs(self.ols_result.rsquared - self.ols_result.rsquared_adj):.4f} <= 0.06, there will not be a problem of over-fitting.')

        ## The F-test of ANOVA
        print('\nAssessment 3: F-test of ANOVA')
        print("H0: All of the estimated coefficients are zero.")
        print("H1: Not all of the estimated coefficients are zero.")
        f_res = self.ols_result.fvalue
        print(f"F value = {f_res:.4f}")
        MSE = self.ols_result.mse_resid
        df_model = self.ols_result.df_model
        df_error = self.ols_result.df_resid
        MSR = f_res * MSE
        SSR = MSR * df_model
        print("SSR = ", SSR, "df = ", df_model, "MSR = ", MSR)
        print("SSE = ", MSE * df_error, "df = ", df_error, "MSE = ", MSE)
        print("F = ", MSR / MSE)
        A = np.identity(len(self.ols_result.params))
        A = A[1:,:]
        print("F test = ", self.ols_result.f_test(A))
        p_value = self.ols_result.f_pvalue
        claim = 'the model is valid'
        my_inference(p_value, alpha, claim)

        # Testing of the Coefficients
        print('\nAssessment 4: t-test of the coefficients')
        for i in range(1, int(self.ols_result.df_model) + 1):
            print(f"H0: beta {i} = 0")
            print(f"H1: beta {i} ≠ 0")
            claim = f"the coefficient of {self.ols_result.params.index[i]} is not 0"
            my_inference(self.ols_result.pvalues[i], self.alpha, claim)

In [ ]:
!gdown 100cvZSLmcYaF6hyRBASBPQI1LSxsb6RP

Downloading...
From: https://drive.google.com/uc?id=100cvZSLmcYaF6hyRBASBPQI1LSxsb6RP
To: /content/nba_player_stats_v2.csv
100% 145k/145k [00:00<00:00, 108MB/s]


In [ ]:
nba_stats = pd.read_csv('./nba_player_stats_v2.csv')
nba_stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,StateLevel,Level1,Level2,PosCtg,Guard,Forward,Salary,lgSalary,gs_rate,Core
0,Jett Howard,SF,20,ORL,18,0,3.7,0.6,1.7,0.333,...,1,1,0,F,0,1,50.26824,15.430299,0.000000,0.000000
1,Tosan Evbuomwan,SF,22,DET,17,8,21.6,2.1,4.2,0.507,...,2,0,1,F,0,1,2.95977,12.598037,0.470588,0.470588
2,Jared Butler,SG,23,WAS,40,0,14.2,2.5,5.0,0.488,...,3,0,0,G,1,0,18.09782,14.408717,0.000000,0.000000
3,Grayson Allen,SG,28,PHO,75,74,33.5,4.5,9.1,0.499,...,2,0,1,G,1,0,89.25000,16.004367,0.986667,0.986667
4,Daniel Theis,C,31,LAC,60,3,16.9,2.6,4.9,0.532,...,1,1,0,C,0,0,91.08387,16.024706,0.050000,0.050000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,Kevin Knox,SF,24,DET,31,11,18.1,2.8,6.0,0.462,...,2,0,1,F,0,1,21.44320,14.578333,0.354839,0.354839
490,KJ Martin,SF,23,PHI,60,2,12.4,1.6,3.0,0.536,...,1,1,0,F,0,1,19.30681,14.473383,0.033333,0.033333
491,Matt Ryan,SF,26,NOP,28,1,13.9,1.8,4.0,0.434,...,3,0,0,F,0,1,20.59782,14.538111,0.035714,0.035714
492,JD Davison,SG,21,BOS,8,0,4.9,0.6,1.5,0.417,...,2,0,1,G,1,0,5.59782,13.235303,0.000000,0.000000


In [ ]:
nba_stats = nba_stats.dropna()

In [ ]:
y = nba_stats['lgSalary']
X = nba_stats.drop(columns = ['Player', 'Pos', 'Age', 'Tm', 'PosCtg', 'lgSalary'])
alpha = 0.05

draft = LAZY_MLR(y, X, alpha)

In [ ]:
draft.get_scatter_plots()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
draft.best_subset()